In [1]:
import pandas as pd
import numpy as np

# Loading data

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
path = '/content/gdrive/MyDrive/Univerui/Kursinis/'

Mounted at /content/gdrive


In [3]:
malmem = pd.read_csv(path + 'Obfuscated-MalMem2022.csv')
malmem.head()

,Category,pslist.nproc,pslist.nppid,pslist.avg_threads,pslist.nprocs64bit,pslist.avg_handlers,dlllist.ndlls,dlllist.avg_dlls_per_proc,handles.nhandles,handles.avg_handles_per_proc,...,svcscan.kernel_drivers,svcscan.fs_drivers,svcscan.process_services,svcscan.shared_process_services,svcscan.interactive_process_services,svcscan.nactive,callbacks.ncallbacks,callbacks.nanonymous,callbacks.ngeneric,Class
0,Benign,45,17,10.555556,0,202.844444,1694,38.500000,9129,212.302326,...,221,26,24,116,0,121,87,0,8,Benign
1,Benign,47,19,11.531915,0,242.234043,2074,44.127660,11385,242.234043,...,222,26,24,118,0,122,87,0,8,Benign
2,Benign,40,14,14.725000,0,288.225000,1932,48.300000,11529,288.225000,...,222,26,27,118,0,120,88,0,8,Benign
3,Benign,32,13,13.500000,0,264.281250,1445,45.156250,8457,264.281250,...,222,26,27,118,0,120,88,0,8,Benign
4,Benign,42,16,11.452381,0,281.333333,2067,49.214286,11816,281.333333,...,222,26,24,118,0,124,87,0,8,Benign


In [4]:
print(f'Columns before dropping {malmem.shape[1]}')
malmem.drop(['pslist.nprocs64bit', 'handles.nport',
             'svcscan.interactive_process_services', 'Category'],
            axis=1, inplace=True)
print(f'Columns after dropping {malmem.shape[1]}')

Columns before dropping 57
Columns after dropping 53


# Data Splitting

In [5]:
entries = malmem.shape[0]
entries

58596

In [40]:
X = malmem.drop(['Class'], axis=1)
y = malmem['Class']

X.shape, y.shape

((58596, 52), (58596,))

In [41]:
from sklearn.model_selection import train_test_split

x, X_test, y, y_test = train_test_split(X, y, test_size=.15, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(x, y, test_size=.178, random_state=42)

In [42]:
X_train.shape[0], X_valid.shape[0], X_test.shape[0]

(40940, 8866, 8790)

In [43]:
print(f'Train dataset contains {X_train.shape[0]/entries:.2f} of whole dataset')
print(f'Validation dataset contains {X_valid.shape[0]/entries:.2f} of whole dataset')
print(f'Test dataset contains {X_test.shape[0]/entries:.2f} of whole dataset')

Train dataset contains 0.70 of whole dataset
Validation dataset contains 0.15 of whole dataset
Test dataset contains 0.15 of whole dataset


In [55]:
cols = X.columns

# Class encoding

In [46]:
y_train[:5]

38718    Malware
58002    Malware
2736      Benign
45808    Malware
15506     Benign
Name: Class, dtype: object

In [47]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_valid = le.transform(y_valid)
y_test = le.transform(y_test)

In [48]:
le.classes_

array(['Benign', 'Malware'], dtype=object)

In [49]:
y_train[:5]

array([1, 1, 0, 1, 0])

# Data standartization

In [50]:
from sklearn.preprocessing import RobustScaler

rs = RobustScaler()

X_train = rs.fit_transform(X_train)
X_valid = rs.transform(X_valid)
X_test = rs.transform(X_test)

# Combining prepared data back to dataframes

In [61]:
train_dataset = pd.DataFrame(X_train, columns=cols)
train_dataset['Class'] = y_train

valid_dataset = pd.DataFrame(X_valid, columns=cols)
valid_dataset['Class'] = y_valid

test_dataset = pd.DataFrame(X_test, columns=cols)
test_dataset['Class'] = y_test

# Saving prepared data

In [63]:
train_dataset.to_csv(path+'train_dataset.csv', index=False)
valid_dataset.to_csv(path+'valid_dataset.csv', index=False)
test_dataset.to_csv(path+'test_dataset.csv', index=False)

In [66]:
train_dataset.shape, valid_dataset.shape, test_dataset.shape

((40940, 53), (8866, 53), (8790, 53))